In [20]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import torchreid
from datetime import datetime
import threading
from collections import defaultdict
import time
import queue
from pathlib import Path
import logging
import csv
import math
from bytetracker import BYTETracker
import torch.nn as nn  # Add this import for neural network components


In [25]:
class PersonTracker:
    def __init__(self):
        # Initialize logging first
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger("PersonTracker")

        self.doors = {
            'camera1': [(1030, 0), (1700, 560)],
            'camera2': [(400, 0), (800, 470)]
        }

        # Define counting zones
        self.counting_zones = {
            'camera1': [(1030, 200), (1700, 300)],
            'camera2': [(400, 200), (800, 300)]
        }

        # Initialize models
        self.yolo_model = self.initialize_yolo()
        self.reid_model = self.initialize_reid()

        # Storage for tracked individuals
        self.tracked_individuals = {}
        self.completed_tracks = set()
        self.current_frame_detections = {}
        self.camera_appearances = defaultdict(dict)

        # Track counts
        self.entry_count = 0
        self.processed_tracks = set()

        # Tracking parameters
        self.reid_threshold = 0.92
        self.max_track_gap = 1.0
        self.min_tracking_frames = 10
        self.confidence_threshold = 5

        # Add new tracking sets
        self.camera1_entries = set()
        self.camera1_to_camera2 = set()

    def initialize_yolo(self):
        """Initialize YOLO model"""
        try:
            model = YOLO("yolov8x.pt")
            return model
        except Exception as e:
            self.logger.error(f"Error loading YOLO model: {e}")
            raise

    def initialize_reid(self):
        """Initialize ReID model"""
        try:
            class OSNet(nn.Module):
                def __init__(self):
                    super(OSNet, self).__init__()
                    # Base convolutional layers
                    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
                    self.bn1 = nn.BatchNorm2d(64)
                    self.relu = nn.ReLU(inplace=True)
                    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
                    
                    # Feature layers
                    self.conv2 = nn.Conv2d(64, 256, kernel_size=3, stride=2, padding=1)
                    self.bn2 = nn.BatchNorm2d(256)
                    self.conv3 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
                    self.bn3 = nn.BatchNorm2d(512)
                    
                    # Global average pooling
                    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
                    
                    # Feature dimension
                    self.feat_dim = 512

                def forward(self, x):
                    x = self.conv1(x)
                    x = self.bn1(x)
                    x = self.relu(x)
                    x = self.maxpool(x)

                    x = self.conv2(x)
                    x = self.bn2(x)
                    x = self.relu(x)

                    x = self.conv3(x)
                    x = self.bn3(x)
                    x = self.relu(x)

                    x = self.avgpool(x)
                    x = x.view(x.size(0), -1)
                    return x

            model = OSNet()
            model.eval()

            if torch.cuda.is_available():
                model = model.cuda()
                self.logger.info("ReID model moved to GPU")
            else:
                self.logger.info("Running ReID model on CPU")

            return model

        except Exception as e:
            self.logger.error(f"Error loading ReID model: {e}")
            raise

    def extract_reid_features(self, frame, bbox):
        """Extract ReID features from detected person"""
        try:
            x1, y1, x2, y2 = bbox
            person_img = frame[y1:y2, x1:x2]
            if person_img.size == 0:
                return None

            person_img = cv2.resize(person_img, (128, 256))
            person_img = cv2.cvtColor(person_img, cv2.COLOR_BGR2RGB)

            # Normalize image
            person_img = person_img.astype(np.float32) / 255.0
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            person_img = (person_img - mean) / std

            # Convert to tensor
            person_img = torch.from_numpy(person_img).permute(2, 0, 1).unsqueeze(0).float()

            if torch.cuda.is_available():
                person_img = person_img.cuda()

            with torch.no_grad():
                features = self.reid_model(person_img)
                features = features.cpu().numpy()

            if features is None or features.size == 0:
                return None

            features = features / np.linalg.norm(features)
            return features

        except Exception as e:
            self.logger.error(f"Error extracting ReID features: {e}")
            return None

    def is_in_door_area(self, bbox, camera_id):
        """Check if detection is in door area"""
        x1, y1, x2, y2 = bbox
        door_coords = self.doors[camera_id]
        
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        
        door_x1, door_y1 = door_coords[0]
        door_x2, door_y2 = door_coords[1]
        
        return (door_x1 <= center_x <= door_x2 and
                door_y1 <= center_y <= door_y2)

    def match_person(self, features, current_time, camera_id):
        """Match person using ReID features"""
        best_match_id = None
        best_match_score = self.reid_threshold

        current_features = np.array(features).flatten()
        current_features = current_features / np.linalg.norm(current_features)

        for person_id, person_info in self.tracked_individuals.items():
            if person_id in self.completed_tracks:
                continue

            if (person_info.last_seen is not None and
                    current_time - person_info.last_seen > self.max_track_gap):
                continue

            for stored_feat in person_info.features:
                score = np.dot(current_features, stored_feat)
                if score > best_match_score:
                    best_match_score = score
                    best_match_id = person_id

        return best_match_id

    def analyze_movement_pattern(self, positions, min_positions=3):
        """Analyze movement pattern"""
        if len(positions) < min_positions:
            return None

        movements = []
        for i in range(1, len(positions)):
            prev_pos = positions[i-1][0]
            curr_pos = positions[i][0]
            dy = curr_pos[1] - prev_pos[1]
            movements.append(dy)

        downward_count = sum(1 for dy in movements if dy > 0)
        if downward_count >= len(movements) * 0.8:
            return 'entering'
        
        return 'other'

    def update_person_info(self, person_id, frame, bbox, camera_id, timestamp, features):
        """Update person information"""
        if person_id not in self.tracked_individuals:
            self.tracked_individuals[person_id] = PersonInfo(person_id)

        person_info = self.tracked_individuals[person_id]
        person_info.update_features(features.squeeze())
        person_info.update_appearance(frame[bbox[1]:bbox[3], bbox[0]:bbox[2]])
        
        if not person_info.update_position(bbox, timestamp):
            return

        person_info.last_camera = camera_id
        person_info.last_seen = timestamp

        if camera_id not in person_info.camera_times:
            person_info.camera_times[camera_id] = {
                'first': timestamp,
                'last': timestamp
            }
        else:
            person_info.camera_times[camera_id]['last'] = timestamp

        movement = self.analyze_movement_pattern(person_info.prev_positions)
        if movement == 'entering' and not person_info.entry_recorded:
            if camera_id == 'camera1':
                self.camera1_entries.add(person_id)
                person_info.entered_camera1 = True
                person_info.camera1_entry_time = timestamp
            elif camera_id == 'camera2' and person_info.entered_camera1:
                self.camera1_to_camera2.add(person_id)
            person_info.entry_recorded = True
            self.entry_count += 1

    def process_frame(self, frame, camera_id, timestamp):
        """Process a single frame"""
        if frame is None:
            return None

        results = self.yolo_model(frame)
        self.current_frame_detections = {}

        for detection in results[0].boxes.data:
            bbox = [int(coord.item()) for coord in detection[:4]]
            confidence = float(detection[4].item())
            class_id = int(detection[5].item())

            if class_id == 0 and confidence > 0.5:
                if self.is_in_door_area(bbox, camera_id):
                    features = self.extract_reid_features(frame, bbox)
                    if features is not None:
                        person_id = self.match_person(features, timestamp, camera_id)

                        if person_id is None:
                            person_id = len(self.tracked_individuals)

                        if person_id not in self.current_frame_detections:
                            self.update_person_info(
                                person_id, frame, bbox, camera_id, timestamp, features)
                            self.current_frame_detections[person_id] = bbox

                            if person_id not in self.completed_tracks:
                                cv2.rectangle(frame, (bbox[0], bbox[1]),
                                            (bbox[2], bbox[3]), (0, 255, 0), 2)
                                cv2.putText(frame, f"ID: {person_id}",
                                          (bbox[0], bbox[1]-10),
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        door_coords = self.doors[camera_id]
        cv2.rectangle(frame,
                     (int(door_coords[0][0]), int(door_coords[0][1])),
                     (int(door_coords[1][0]), int(door_coords[1][1])),
                     (255, 0, 255), 2)

        cv2.putText(frame, f"Valid Entries: {self.entry_count}", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        return frame

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 52)

In [26]:
class PersonInfo:
    def __init__(self, person_id):
        self.person_id = person_id
        self.appearances = []
        self.features = []  # Store multiple features
        self.prev_positions = []
        self.last_position = None
        self.last_seen = None
        self.last_camera = None

        # Entry/Exit tracking
        self.entry_recorded = False
        self.exit_recorded = False
        self.entered_camera1 = False
        self.has_exited_camera1 = False
        self.camera1_entry_time = None
        self.camera1_exit_time = None
        self.camera_times = {}  # Store timestamps for each camera

    def update_appearance(self, image):
        """Store appearance image"""
        if image.size > 0:  # Only store valid images
            self.appearances.append(image.copy())
            if len(self.appearances) > 10:  # Keep last 10 appearances
                self.appearances.pop(0)

    def update_features(self, new_features):
        """Store multiple features for better matching"""
        feat = np.array(new_features).flatten()
        feat = feat / np.linalg.norm(feat)  # Normalize feature vector
        self.features.append(feat)
        if len(self.features) > 10:  # Keep last 10 features
            self.features.pop(0)

    def update_position(self, bbox, timestamp):
        """Update position with timestamp"""
        center = ((bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2)
        self.prev_positions.append((center, timestamp))
        if len(self.prev_positions) > 30:  # Keep last 30 positions
            self.prev_positions.pop(0)
        self.last_position = center
        self.last_seen = timestamp
        return True

In [27]:
tracker = PersonTracker()
video_dir = os.path.join('C:\\Users', 'mc1159', 'OneDrive - University of Exeter',
                         'Documents', 'VISIONARY', 'Durham Experiment', 'Experiment Data', 'Before')

# video_dir = os.path.join('C:\\Users', 'mc1159', 'OneDrive - University of Exeter',
#                          'Documents', 'VISIONARY', 'Durham Experiment', 'test_data')

try:
    tracker.process_videos(video_dir)
    results = tracker.analyze_tracks()

    print("\nTracking Results:")
    print(f"Total unique individuals: {results['total_unique_individuals']}")
    print(
        f"People moving from Camera 1 to Camera 2: {results['camera1_to_camera2_count']}")

except Exception as e:
    logging.error(f"Error during tracking: {e}")
    raise

INFO:PersonTracker:ReID model moved to GPU
INFO:PersonTracker:
Processing videos for date: 20241011
INFO:PersonTracker:Processing C:\Users\mc1159\OneDrive - University of Exeter\Documents\VISIONARY\Durham Experiment\Experiment Data\Before\Camera_1_20241011.mp4
ERROR:root:Error during tracking: 'PersonTracker' object has no attribute 'process_frame'


AttributeError: 'PersonTracker' object has no attribute 'process_frame'

# Backup

In [37]:
class PersonTracker:
    def __init__(self):
        # Initialize logging first
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger("PersonTracker")

        self.doors = {
            'camera1': [(1030, 0), (1700, 560)],
            'camera2': [(400, 0), (800, 470)]
        }

        # Define counting zones
        self.counting_zones = {
            'camera1': [(1030, 200), (1700, 300)],
            'camera2': [(400, 200), (800, 300)]
        }

        try:
            # Initialize YOLO model
            self.yolo_model = YOLO("yolov8x.pt")  # Using YOLOv8 instead of YOLO11
            self.logger.info("YOLO model initialized successfully")

            # Initialize OSNet for ReID
            class OSNet(nn.Module):
                def __init__(self):
                    super(OSNet, self).__init__()
                    # Base convolutional layers
                    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
                    self.bn1 = nn.BatchNorm2d(64)
                    self.relu = nn.ReLU(inplace=True)
                    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
                    
                    # Feature layers
                    self.conv2 = nn.Conv2d(64, 256, kernel_size=3, stride=2, padding=1)
                    self.bn2 = nn.BatchNorm2d(256)
                    self.conv3 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
                    self.bn3 = nn.BatchNorm2d(512)
                    
                    # Global average pooling
                    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
                    
                    # Feature dimension
                    self.feat_dim = 512

                def forward(self, x):
                    x = self.conv1(x)
                    x = self.bn1(x)
                    x = self.relu(x)
                    x = self.maxpool(x)

                    x = self.conv2(x)
                    x = self.bn2(x)
                    x = self.relu(x)

                    x = self.conv3(x)
                    x = self.bn3(x)
                    x = self.relu(x)

                    x = self.avgpool(x)
                    x = x.view(x.size(0), -1)
                    return x

            # Initialize and set up the ReID model
            self.reid_model = OSNet()
            self.reid_model.eval()  # Set to evaluation mode

            # Move to GPU if available
            if torch.cuda.is_available():
                self.reid_model = self.reid_model.cuda()
                self.logger.info("ReID model moved to GPU")
            else:
                self.logger.info("Running ReID model on CPU")

        except Exception as e:
            self.logger.error(f"Error initializing models: {e}")
            raise

        # Storage for tracked individuals
        self.tracked_individuals = {}
        self.completed_tracks = set()
        self.current_frame_detections = {}
        self.camera_appearances = defaultdict(dict)

        # Track counts
        self.entry_count = 0
        self.processed_tracks = set()

        # Tracking parameters
        self.reid_threshold = 0.92
        self.max_track_gap = 1.0
        self.min_tracking_frames = 10
        self.confidence_threshold = 5

        # Add new tracking sets
        self.camera1_entries = set()
        self.camera1_to_camera2 = set()

    def extract_reid_features(self, frame, bbox):
        """Extract ReID features from detected person"""
        try:
            x1, y1, x2, y2 = bbox
            person_img = frame[y1:y2, x1:x2]
            if person_img.size == 0:
                return None

            person_img = cv2.resize(person_img, (128, 256))
            person_img = cv2.cvtColor(person_img, cv2.COLOR_BGR2RGB)

            # Normalize image
            person_img = person_img.astype(np.float32) / 255.0
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            person_img = (person_img - mean) / std

            # Convert to tensor
            person_img = torch.from_numpy(person_img).permute(2, 0, 1).unsqueeze(0).float()

            if torch.cuda.is_available():
                person_img = person_img.cuda()

            with torch.no_grad():
                features = self.reid_model(person_img)
                features = features.cpu().numpy()

            if features is None or features.size == 0:
                return None

            features = features / np.linalg.norm(features)
            return features

        except Exception as e:
            self.logger.error(f"Error extracting ReID features: {e}")
            return None

    def is_in_door_area(self, bbox, camera_id):
        """Check if detection is in door area"""
        x1, y1, x2, y2 = bbox
        door_coords = self.doors[camera_id]
        
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        
        door_x1, door_y1 = door_coords[0]
        door_x2, door_y2 = door_coords[1]
        
        return (door_x1 <= center_x <= door_x2 and
                door_y1 <= center_y <= door_y2)

    def match_person(self, features, current_time, camera_id):
        """Match person using ReID features"""
        best_match_id = None
        best_match_score = self.reid_threshold

        current_features = np.array(features).flatten()
        current_features = current_features / np.linalg.norm(current_features)

        for person_id, person_info in self.tracked_individuals.items():
            if person_id in self.completed_tracks:
                continue

            if (person_info.last_seen is not None and
                    current_time - person_info.last_seen > self.max_track_gap):
                continue

            for stored_feat in person_info.features:
                score = np.dot(current_features, stored_feat)
                if score > best_match_score:
                    best_match_score = score
                    best_match_id = person_id

        return best_match_id

    def analyze_movement_pattern(self, positions, min_positions=3):
        """Analyze movement pattern"""
        if len(positions) < min_positions:
            return None

        movements = []
        for i in range(1, len(positions)):
            prev_pos = positions[i-1][0]
            curr_pos = positions[i][0]
            dy = curr_pos[1] - prev_pos[1]
            movements.append(dy)

        downward_count = sum(1 for dy in movements if dy > 0)
        if downward_count >= len(movements) * 0.8:
            return 'entering'
        
        return 'other'

    def update_person_info(self, person_id, frame, bbox, camera_id, timestamp, features):
        """Update person information"""
        if person_id not in self.tracked_individuals:
            self.tracked_individuals[person_id] = PersonInfo(person_id)

        person_info = self.tracked_individuals[person_id]
        person_info.update_features(features.squeeze())
        person_info.update_appearance(frame[bbox[1]:bbox[3], bbox[0]:bbox[2]])
        
        if not person_info.update_position(bbox, timestamp):
            return

        person_info.last_camera = camera_id
        person_info.last_seen = timestamp

        if camera_id not in person_info.camera_times:
            person_info.camera_times[camera_id] = {
                'first': timestamp,
                'last': timestamp
            }
        else:
            person_info.camera_times[camera_id]['last'] = timestamp

        movement = self.analyze_movement_pattern(person_info.prev_positions)
        if movement == 'entering' and not person_info.entry_recorded:
            if camera_id == 'camera1':
                self.camera1_entries.add(person_id)
                person_info.entered_camera1 = True
                person_info.camera1_entry_time = timestamp
            elif camera_id == 'camera2' and person_info.entered_camera1:
                self.camera1_to_camera2.add(person_id)
            person_info.entry_recorded = True
            self.entry_count += 1

    def process_frame(self, frame, camera_id, timestamp):
        """Process a single frame"""
        if frame is None:
            return None

        results = self.yolo_model(frame)
        self.current_frame_detections = {}

        for detection in results[0].boxes.data:
            bbox = [int(coord.item()) for coord in detection[:4]]
            confidence = float(detection[4].item())
            class_id = int(detection[5].item())

            if class_id == 0 and confidence > 0.5:
                if self.is_in_door_area(bbox, camera_id):
                    features = self.extract_reid_features(frame, bbox)
                    if features is not None:
                        person_id = self.match_person(features, timestamp, camera_id)

                        if person_id is None:
                            person_id = len(self.tracked_individuals)

                        if person_id not in self.current_frame_detections:
                            self.update_person_info(
                                person_id, frame, bbox, camera_id, timestamp, features)
                            self.current_frame_detections[person_id] = bbox

                            if person_id not in self.completed_tracks:
                                cv2.rectangle(frame, (bbox[0], bbox[1]),
                                            (bbox[2], bbox[3]), (0, 255, 0), 2)
                                cv2.putText(frame, f"ID: {person_id}",
                                          (bbox[0], bbox[1]-10),
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        door_coords = self.doors[camera_id]
        cv2.rectangle(frame,
                     (int(door_coords[0][0]), int(door_coords[0][1])),
                     (int(door_coords[1][0]), int(door_coords[1][1])),
                     (255, 0, 255), 2)

        cv2.putText(frame, f"Valid Entries: {self.entry_count}", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        return frame

    def process_videos(self, video_dir, output_dir=None):
        """Process videos grouped by date"""
        if output_dir is None:
            output_dir = os.path.join(video_dir, 'tracking_results')

        videos_by_date = defaultdict(list)
        for video_file in Path(video_dir).glob("Camera_*_*.mp4"):
            date = self.extract_date_from_filename(video_file)
            if date:
                videos_by_date[date].append(video_file)

        for date, video_files in videos_by_date.items():
            self.reset_tracking()
            self.logger.info(f"\nProcessing videos for date: {date}")

            for video_file in sorted(video_files):
                camera_id = "camera1" if "Camera_1" in str(video_file) else "camera2"
                self.logger.info(f"Processing {video_file}")

                cap = cv2.VideoCapture(str(video_file))
                if not cap.isOpened():
                    self.logger.error(f"Error opening video file: {video_file}")
                    continue

                fps = int(cap.get(cv2.CAP_PROP_FPS))
                frame_count = 0

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    frame_count += 1
                    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
                    processed_frame = self.process_frame(frame, camera_id, timestamp)

                    cv2.imshow(f"Camera {camera_id}", processed_frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                    if frame_count % 100 == 0:
                        self.logger.info(f"Processed {frame_count} frames from {camera_id}")

                cap.release()
                cv2.destroyWindow(f"Camera {camera_id}")

            if output_dir:
                self.save_tracking_data(output_dir, date)

        cv2.destroyAllWindows()

    def extract_date_from_filename(self, filename):
        """Extract date from filename format Camera_X_YYYYMMDD"""
        try:
            date_str = str(filename).split('_')[-1].split('.')[0]
            return date_str
        except:
            return None

    def analyze_tracks(self):
        """Analyze tracking results"""
        results = {
            'total_unique_individuals': len(self.tracked_individuals) - len(self.completed_tracks),
            'total_entries': self.entry_count,
            'camera1_entries': len(self.camera1_entries),
            'camera2_entries': len(set(pid for pid, info in self.tracked_individuals.items()
                                   if hasattr(info, 'camera_times') and 'camera2' in info.camera_times)),
            'camera1_to_camera2_count': len(self.camera1_to_camera2),
            'camera1_to_camera2_ids': list(self.camera1_to_camera2),
            'transitions': []
        }

        for pid in self.camera1_to_camera2:
            if pid in self.tracked_individuals:
                person_info = self.tracked_individuals[pid]
                if hasattr(person_info, 'camera_times'):
                    camera1_times = person_info.camera_times.get('camera1', {})
                    camera2_times = person_info.camera_times.get('camera2', {})
                    
                    if camera1_times and camera2_times:
                        results['transitions'].append({
                            'person_id': pid,
                            'camera1_exit': camera1_times.get('last'),
                            'camera2_entry': camera2_times.get('first')
                        })

        valid_transitions = [t for t in results['transitions']
                           if t['camera1_exit'] is not None and t['camera2_entry'] is not None]
        
        if valid_transitions:
            transit_times = [(t['camera2_entry'] - t['camera1_exit'])
                           for t in valid_transitions]
            results['average_transit_time'] = sum(transit_times) / len(transit_times)

        return results

    def reset_tracking(self):
        """Reset tracking states for new date"""
        self.tracked_individuals.clear()
        self.completed_tracks.clear()
        self.current_frame_detections.clear()
        self.camera1_entries.clear()
        self.camera1_to_camera2.clear()
        self.entry_count = 0

    def save_tracking_data(self, output_dir, date):
        """Save tracking data to CSV files"""
        os.makedirs(output_dir, exist_ok=True)

        # Save entries data
        entries_file = os.path.join(output_dir, f'entries_{date}.csv')
        with open(entries_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Date', 'Person_ID', 'Camera_ID', 'Entry_Time', 'Exit_Time'])
            
            for person_id, person_info in self.tracked_individuals.items():
                if hasattr(person_info, 'camera_times'):
                    for camera_id, times in person_info.camera_times.items():
                        writer.writerow([
                            date,
                            person_id,
                            camera_id,
                            f"{times.get('first', ''):.2f}" if times.get('first') else '',
                            f"{times.get('last', ''):.2f}" if times.get('last') else ''
                        ])

        # Save transitions data
        transitions_file = os.path.join(output_dir, f'camera_transitions_{date}.csv')
        with open(transitions_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Date', 'Person_ID', 'Camera1_Exit', 'Camera2_Entry', 'Transit_Time_Seconds'])
            
            for pid in self.camera1_to_camera2:
                person_info = self.tracked_individuals.get(pid)
                if person_info and hasattr(person_info, 'camera_times'):
                    camera1_times = person_info.camera_times.get('camera1', {})
                    camera2_times = person_info.camera_times.get('camera2', {})
                    
                    camera1_exit = camera1_times.get('last')
                    camera2_entry = camera2_times.get('first')
                    
                    if camera1_exit and camera2_entry:
                        transit_time = camera2_entry - camera1_exit
                        writer.writerow([
                            date,
                            pid,
                            f"{camera1_exit:.2f}",
                            f"{camera2_entry:.2f}",
                            f"{transit_time:.2f}"
                        ])

        # Save summary statistics
        summary_file = os.path.join(output_dir, f'tracking_summary_{date}.csv')
        with open(summary_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Date', 'Metric', 'Value'])
            writer.writerow([date, 'Total_Camera1_Entries', len(self.camera1_entries)])
            writer.writerow([
                date, 
                'Total_Camera2_Entries',
                len(set(pid for pid, info in self.tracked_individuals.items()
                    if hasattr(info, 'camera_times') and 'camera2' in info.camera_times))
            ])
            writer.writerow([date, 'Camera1_to_Camera2_Transitions', len(self.camera1_to_camera2)])

In [38]:
def process_videos(video_dir):
    tracker = PersonTracker()

    # Get all video files and sort them by date
    video_files = sorted([f for f in os.listdir(video_dir)
                         if f.startswith(('Camera_1_', 'Camera_2_'))])

    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        # Determine camera ID from filename
        camera_id = 'camera1' if video_file.startswith(
            'Camera_1_') else 'camera2'

        print(f"\nProcessing {video_file}...")
        tracker.process_video(video_path, camera_id)

    # Save results
    tracker.save_person_images('person_images')
    stats = tracker.get_statistics()

    # Print statistics
    print("\nTracking Statistics:")
    print(f"Total unique individuals: {stats['total_unique_individuals']}")
    print(f"Individuals in Camera 1: {stats['camera1_count']}")
    print(f"Individuals in Camera 2: {stats['camera2_count']}")
    print(
        f"Individuals appearing in both cameras: {stats['camera1_to_camera2']}")

    cv2.destroyAllWindows()

In [36]:
# Process videos from both cameras
video_dir = os.path.join('C:\\Users', 'mc1159', 'OneDrive - University of Exeter',
                         'Documents', 'VISIONARY', 'Durham Experiment', 'test_data')


process_videos(video_dir)

INFO:PersonTracker:YOLO model initialized successfully
INFO:PersonTracker:ReID model moved to GPU



Processing Camera_1_20241011.mp4...


AttributeError: 'PersonTracker' object has no attribute 'process_video'